<a href="https://colab.research.google.com/github/JesseKullar/SMS-Spam-Detector/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
MAXLEN = 900 # length of longest message is 910 characters
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 1000
unknown_token = '[UNK]'
EPOCHS=10

In [ ]:
label = {'ham': 0, 'spam': 1}

train_dataset = pd.read_csv(train_file_path, sep='\t', names=['label', 'message'])
test_dataset = pd.read_csv(test_file_path, sep='\t', names=['label', 'message'])

train_dataset['label'] = [label[i] for i in train_dataset['label']]
test_dataset['label'] = [label[i] for i in test_dataset['label']]

train_labels = train_dataset.pop('label')
test_labels = test_dataset.pop('label')


In [ ]:
# Encode text and make each message the same length
encoder = keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE,
                                             char_level=False,
                                             oov_token = unknown_token)
encoder.fit_on_texts(train_dataset['message'])
vocab = encoder.word_index


def encode_text(text):
  sequence = encoder.texts_to_sequences(text)
  return keras.preprocessing.sequence.pad_sequences(sequence, MAXLEN)


train_data_enc = encode_text(train_dataset['message'])
test_data_enc = encode_text(test_dataset['message'])


In [ ]:
# Build Model
model = tf.keras.Sequential([tf.keras.layers.Embedding(VOCAB_SIZE, 64),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dense(32, activation='relu'),
                             tf.keras.layers.Dropout(rate=0.2),
                             tf.keras.layers.Dense(1, activation='sigmoid')])
model.summary()

# Compile Model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [ ]:
# Train Model
history = model.fit(train_data_enc, 
                    train_labels,
                    validation_data=(test_data_enc, test_labels), 
                    epochs=EPOCHS,
                    verbose=2)

In [ ]:
# Evaluate with Test Data
results = model.evaluate(test_data_enc, test_labels)
print(results)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = [pred_text]
  pred_text_enc = encode_text(pred)
  prediction = model.predict(pred_text_enc)[0][0]
  for key, value in label.items():
    if round(prediction) == value:
      pred_label = key
  return ([prediction, pred_label])

pred_text = "how are you doing today?"


prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True   

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
